In [115]:
# Imports
import pandas as pd

In [163]:
# Load in data

# Encoding is specified for each import to resolve an "unexpected continuation byte" error

# https://www.kaggle.com/code/nelgiriyewithana/introduction-to-world-educational-data/notebook
world_education_data = pd.read_csv("Data/Global_Education.csv", encoding="iso-8859-1")

# https://databank.worldbank.org/source/education-statistics-%5E-all-indicators
world_bank_education_data = pd.read_csv("Data/World Bank Education Data.csv", encoding="ascii")

# The following are from the UNESCO Institute for Statistics Data Browser for Education available here: https://databrowser.uis.unesco.org/browser/EDUCATION/UIS-SDG4Monitoring
teacher_data = pd.read_csv("Data/UIS Teacher Data/data.csv")
education_years_data = pd.read_csv("Data/UIS Years of Education/data.csv")
teacher_attrition_data = pd.read_csv("Data/UIS Teacher Attrition/data.csv")

In [117]:
#UIS Data

#Dropping values we do not need
teacher_data = teacher_data.drop(["qualifier", "magnitude"], axis=1)
education_years_data = education_years_data.drop(["qualifier", "magnitude"], axis=1)
teacher_attrition_data = teacher_attrition_data.drop(["qualifier", "magnitude"], axis=1)

In [118]:
# Work out which year has the most complete data
print((teacher_data["year"].value_counts() + education_years_data["year"].value_counts() + teacher_attrition_data["year"].value_counts()).sort_values())

year
2024     150.0
2011     663.0
2012     691.0
2013     906.0
2014    1035.0
2015    1090.0
2017    1119.0
2016    1120.0
2018    1143.0
2020    1147.0
2023    1161.0
2019    1165.0
2022    1166.0
2021    1199.0
2010       NaN
Name: count, dtype: float64


In [119]:
#2021 is the most complete year, so we will use it
years = range(2010, 2022)

new_data = pd.DataFrame(columns=["indicator", "country"])

for i in years: 
    new_data[str(i)] = []

In [120]:
#Data in the files is sorted by indicatorId which makes it easier to add them to our new_data dataframe

def load_into_data(data: pd.DataFrame, dataset: pd.DataFrame) -> pd.DataFrame:
    for index, _ in dataset.iterrows():
        if index == 0:
            lastId = None
            lastGeo = None
        else:
            lastId = dataset.iloc[index -1]["indicatorId"]
            lastGeo = dataset.iloc[index -1]["geoUnit"]

        currId = dataset.iloc[index]["indicatorId"]
        currGeo = dataset.iloc[index]["geoUnit"]
        year = dataset.iloc[index]["year"]
        
        if year > 2021:
            continue

        if lastId != currId or lastGeo != currGeo:
            df = pd.DataFrame({"indicator": [currId], "country": [currGeo]})
            data = pd.concat([data, df], ignore_index=True)
        data.at[len(data) -1, str(year)] = dataset.iloc[index]["value"]
    return data

new_data = load_into_data(new_data,teacher_data)
new_data = load_into_data(new_data,education_years_data)
new_data = load_into_data(new_data,teacher_attrition_data)



In [121]:
data = new_data.copy(deep=True)

In [159]:
def get_average_for_row(row: pd.Series) -> float:
    total = 0
    count = 0
    for cell in row:
        try:
            cell = float(cell)
        except ValueError:
            continue
        if math.isnan(cell):
            continue
        total += cell
        count += 1

    if count == 0:
        return 0
    
    return total / count

In [ ]:
# Replacing missing values

import math
# Fill in missing 2021 values with the average of previous years

# Get which indicies are missing a value of 2021.
indexes_to_fill = []
for index, row in data.iterrows():
    if math.isnan(row["2021"]):
        indexes_to_fill.append(index)

print(indexes_to_fill)

for i in indexes_to_fill:
    data.at[i, "2021"] = get_average_for_row(data.iloc[i])

data
    

[6, 9, 13, 17, 18, 19, 22, 26, 27, 28, 29, 32, 36, 40, 42, 45, 48, 53, 54, 55, 56, 58, 59, 61, 63, 64, 68, 69, 71, 72, 74, 76, 78, 82, 84, 86, 87, 88, 89, 94, 96, 97, 98, 101, 103, 107, 110, 112, 115, 118, 121, 125, 127, 133, 135, 139, 143, 148, 149, 150, 153, 159, 166, 169, 171, 174, 179, 180, 186, 190, 194, 195, 197, 201, 206, 207, 210, 212, 213, 215, 219, 222, 224, 227, 228, 229, 230, 236, 239, 241, 244, 246, 247, 248, 251, 254, 257, 260, 263, 264, 268, 272, 273, 275, 281, 283, 287, 291, 296, 299, 300, 303, 305, 314, 316, 317, 322, 328, 332, 334, 335, 337, 338, 343, 344, 345, 346, 347, 348, 349, 350, 351, 353, 354, 355, 357, 359, 361, 362, 363, 371, 372, 375, 376, 377, 378, 379, 380, 381, 383, 388, 389, 393, 396, 399, 400, 406, 408, 411, 415, 419, 420, 421, 422, 425, 429, 430, 431, 432, 435, 439, 441, 444, 446, 449, 453, 458, 459, 460, 461, 463, 464, 466, 468, 469, 473, 474, 476, 477, 479, 481, 483, 487, 489, 491, 492, 493, 494, 499, 501, 502, 503, 508, 512, 515, 517, 520, 523, 526,

,indicator,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,PTRHC.02.QUALIFIED,AGO,NaN,NaN,NaN,NaN,NaN,NaN,88.033783,NaN,NaN,NaN,NaN,67.013420
1,PTRHC.02.QUALIFIED,ALB,NaN,NaN,NaN,28.42535,25.840099,24.001751,22.056330,20.786989,23.51306,24.01643,20.371010,20.236031
2,PTRHC.02.QUALIFIED,AND,NaN,NaN,NaN,14.08721,13.703910,13.408840,13.684210,13.883440,13.27711,12.56213,12.051720,11.747060
3,PTRHC.02.QUALIFIED,ARE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.26619,26.76902,23.275150,21.276951
4,PTRHC.02.QUALIFIED,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.27090,6.20682,6.918270,4.534200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,TATTRR.2T3,TCA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.879030
1597,TATTRR.2T3,TKL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1598,TATTRR.2T3,TUV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.619049,1.515150
1599,TATTRR.2T3,UZB,NaN,NaN,NaN,NaN,NaN,NaN,3.150820,NaN,NaN,NaN,NaN,3.150820


In [123]:
# Converting country codes to names
import pycountry

for index, _ in data.iterrows():
    data.at[index, "country"] = pycountry.countries.get(alpha_3 = data.at[index, "country"]).name

data

,indicator,country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,PTRHC.02.QUALIFIED,Angola,NaN,NaN,NaN,NaN,NaN,NaN,88.033783,NaN,NaN,NaN,NaN,67.013420
1,PTRHC.02.QUALIFIED,Albania,NaN,NaN,NaN,28.42535,25.840099,24.001751,22.056330,20.786989,23.51306,24.01643,20.371010,20.236031
2,PTRHC.02.QUALIFIED,Andorra,NaN,NaN,NaN,14.08721,13.703910,13.408840,13.684210,13.883440,13.27711,12.56213,12.051720,11.747060
3,PTRHC.02.QUALIFIED,United Arab Emirates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.26619,26.76902,23.275150,21.276951
4,PTRHC.02.QUALIFIED,Armenia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.27090,6.20682,6.918270,4.534200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596,TATTRR.2T3,Turks and Caicos Islands,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.879030
1597,TATTRR.2T3,Tokelau,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000
1598,TATTRR.2T3,Tuvalu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.619049,1.515150
1599,TATTRR.2T3,Uzbekistan,NaN,NaN,NaN,NaN,NaN,NaN,3.150820,NaN,NaN,NaN,NaN,3.150820


In [124]:
# Removing all but 2021

data = data.drop(["2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020"], axis=1)
data

,indicator,country,2021
0,PTRHC.02.QUALIFIED,Angola,67.013420
1,PTRHC.02.QUALIFIED,Albania,20.236031
2,PTRHC.02.QUALIFIED,Andorra,11.747060
3,PTRHC.02.QUALIFIED,United Arab Emirates,21.276951
4,PTRHC.02.QUALIFIED,Armenia,4.534200
...,...,...,...
1596,TATTRR.2T3,Turks and Caicos Islands,9.879030
1597,TATTRR.2T3,Tokelau,0.000000
1598,TATTRR.2T3,Tuvalu,1.515150
1599,TATTRR.2T3,Uzbekistan,3.150820


In [125]:
columns = ["country"]
columns.extend(data["indicator"].unique())

final_data = pd.DataFrame(columns=columns)

final_data

,country,PTRHC.02.QUALIFIED,PTRHC.1.QUALIFIED,PTRHC.2T3.QUALIFIED,QUTP.02,QUTP.1,QUTP.2T3,YEARS.FC.COMP.1T3,YEARS.FC.FREE.02,YEARS.FC.FREE.1T3,TATTRR.02,TATTRR.1,TATTRR.2T3


In [126]:
for index, _ in data.iterrows():
    country = data.at[index, "country"]
    indicator = data.at[index, "indicator"]
    indexes: list = final_data[final_data["country"] == country].index.tolist()

    if len(indexes) == 0:
        final_data.loc[len(final_data)] = pd.Series()
        final_data.at[final_data.index[-1], "country"] = country
        indexes.append(len(final_data) -1)
    final_data.at[indexes[0], indicator] = data.iloc[index]["2021"]
    

In [164]:
# World Bank Education data
world_bank_education_data = world_bank_education_data.drop(world_bank_education_data.columns[[1, 3]], axis=1)

for index, row in world_bank_education_data.iterrows():
    if world_bank_education_data.loc[index]["2021 [YR2021]"] == "..":
        world_bank_education_data.loc[index]["2021 [YR2021]"] = get_average_for_row(world_bank_education_data.loc[index])

world_bank_education_data

,Country Name,Series,2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],...,2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023],2024 [YR2024]
0,Afghanistan,Human Capital Index (HCI): Expected Years of S...,..,..,..,..,..,..,..,..,...,..,..,9.21,..,..,..,9.21,..,..,..
1,Afghanistan,Human Capital Index (HCI): Expected Years of S...,..,..,..,..,..,..,..,..,...,..,..,6.73,..,..,..,6.73,..,..,..
2,Afghanistan,Human Capital Index (HCI): Expected Years of S...,..,..,..,..,..,..,..,..,...,..,..,8.58,..,..,..,8.58,..,..,..
3,Afghanistan,Government expenditure on secondary education ...,..,..,..,..,..,..,..,..,...,0.79299,0.92321,0.90325,..,..,..,0.870019,..,..,..
4,Afghanistan,Government expenditure on tertiary education a...,..,..,..,..,..,..,..,..,...,0.51942,..,..,..,..,..,0.390228,..,..,..
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1627,Zimbabwe,Human Capital Index (HCI): Expected Years of S...,..,..,..,..,..,..,..,..,...,..,..,9.99,..,..,..,9.99,..,..,..
1628,Zimbabwe,Human Capital Index (HCI): Expected Years of S...,..,..,..,..,..,..,..,..,...,..,..,10.01,..,..,..,10.01,..,..,..
1629,Zimbabwe,Government expenditure on secondary education ...,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,1.305727,..,..,..
1630,Zimbabwe,Government expenditure on tertiary education a...,..,..,..,..,..,..,..,..,...,..,..,..,..,..,..,0.788906,..,..,..


In [ ]:
world_bank_education_data.drop(world_bank_education_data.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,-1, -2, -3]], axis = 1)

,Country Name,Series,2021 [YR2021]
0,Afghanistan,Human Capital Index (HCI): Expected Years of S...,9.21
1,Afghanistan,Human Capital Index (HCI): Expected Years of S...,6.73
2,Afghanistan,Human Capital Index (HCI): Expected Years of S...,8.58
3,Afghanistan,Government expenditure on secondary education ...,0.870019
4,Afghanistan,Government expenditure on tertiary education a...,0.390228
...,...,...,...
1627,Zimbabwe,Human Capital Index (HCI): Expected Years of S...,9.99
1628,Zimbabwe,Human Capital Index (HCI): Expected Years of S...,10.01
1629,Zimbabwe,Government expenditure on secondary education ...,1.305727
1630,Zimbabwe,Government expenditure on tertiary education a...,0.788906


In [179]:
world_bank_education_data.rename({"Country Name": "country"})
columns = ["country"]
columns.extend(world_bank_education_data["Series"].unique())
final_world_bank_education_data = pd.DataFrame(columns=columns)

for index, _ in world_bank_education_data.iterrows():
    country = world_bank_education_data.at[index, "Country Name"]
    indicator = world_bank_education_data.at[index, "Series"]
    indexes: list = final_world_bank_education_data[final_world_bank_education_data["country"] == country].index.to_list()

    if len(indexes) == 0:
        final_world_bank_education_data.loc[len(final_world_bank_education_data)] = pd.Series()
        final_world_bank_education_data.at[final_world_bank_education_data.index[-1], "country"] = country
        indexes.append(len(final_world_bank_education_data) -1)
    final_world_bank_education_data.at[indexes[0], indicator] = world_bank_education_data.iloc[index]["2021 [YR2021]"]

In [166]:
# Joining data together

In [167]:
# Multivariate Analysis

In [168]:
# Normalisation

In [169]:
# Weighting and Aggregation

In [170]:
# Visualisation of Results